In [1]:
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

import ast
from collections import defaultdict
import numpy as np

/usr/local/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Modeling: Transfer learning en ML

In [2]:
models = {
    'SVR': SVR(),
    'KNeighborsRegressor': KNeighborsRegressor(),
    'Lasso': Lasso(),
    'LinearRegression': LinearRegression(),
    'KNN': KNeighborsRegressor(),
    'RandomForest': RandomForestRegressor(),
    'GradientBoosting': GradientBoostingRegressor(),
    'AdaBoost': AdaBoostRegressor(),
    'XGBoost': XGBRegressor()
}

In [3]:
bests_models = pd.read_excel('../data/final_data/bests_models_results.xlsx')
bests_models

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.829683,2326.718154,2683.106973
1,new_values,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.878805,12267.590107,15930.705726
2,new_values,iter2-cov,NC,LinearRegression,{},0.911957,10052.628324,12590.826233
3,all_pca,iter2-cov,NJ,LinearRegression,{},0.595462,31778.200052,35635.501147
4,new_values,iter1-normal,NY,GradientBoosting,{'model__n_estimators': 150},0.912735,80.496891,99.511838
5,new_values,iter2-cov,SC,Lasso,{'model__alpha': 1.0},0.835408,8479.319918,10491.318644
6,new_values,iter2-cov,VA,LinearRegression,{},0.921119,12934.661538,16856.425698


In [4]:
mejores_modelos = bests_models[['State','Modelo','Parameters','Version_data']].to_dict(orient='records')

modelos_dict = {}
for diccionario in mejores_modelos:
    modelos_dict[diccionario.get('State')] = {'Modelo': diccionario.get('Modelo'), 'Parameters': diccionario.get('Parameters'),
    'Version_data': diccionario.get('Version_data')}
modelos_dict

{'FL': {'Modelo': 'GradientBoosting',
  'Parameters': "{'model__n_estimators': 50}",
  'Version_data': 'new_values_log'},
 'GA': {'Modelo': 'Lasso',
  'Parameters': "{'model__alpha': 1.0}",
  'Version_data': 'new_values'},
 'NC': {'Modelo': 'LinearRegression',
  'Parameters': '{}',
  'Version_data': 'new_values'},
 'NJ': {'Modelo': 'LinearRegression',
  'Parameters': '{}',
  'Version_data': 'all_pca'},
 'NY': {'Modelo': 'GradientBoosting',
  'Parameters': "{'model__n_estimators': 150}",
  'Version_data': 'new_values'},
 'SC': {'Modelo': 'Lasso',
  'Parameters': "{'model__alpha': 1.0}",
  'Version_data': 'new_values'},
 'VA': {'Modelo': 'LinearRegression',
  'Parameters': '{}',
  'Version_data': 'new_values'}}

# Aumented data OverSampling

In [8]:
results_dict = {
    'Version_data': [],
    'Version_iteracion': 'iter3-OverSampling',
    'State': [],
    'Modelo': [],
    'Parameters': [],
    'R2': [],
    'MAE': [],
    'RMSE': []
}
States = bests_models.State.unique()
directory = '../data/final_data/'

for state in States:
    results_dict['Version_data'].append(modelos_dict[state].get('Version_data'))
    results_dict['Modelo'].append(modelos_dict[state].get('Modelo'))
    results_dict['Parameters'].append(modelos_dict[state].get('Parameters'))
    results_dict['State'].append(state)

    filename = f"data_{state}.csv"
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
        X = data.drop(['MedianPrice', 'State'], axis=1)
        y = data['MedianPrice'] 

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        
        # Sobremuestreo utilizando Random Oversampling
        ros = RandomOverSampler()
        X_train, y_train = ros.fit_resample(X_train, y_train)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        model_name = modelos_dict[state].get('Modelo')
        model = models.get(model_name)

        if model is not None:
            parametros = modelos_dict[state].get('Parameters')

            if isinstance(model, type):
                model.set_params(**parametros)

            # Entrenar el modelo y hacer predicciones
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calcular scores
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred, squared=False)

            results_dict['R2'].append(r2)
            results_dict['MAE'].append(mae)
            results_dict['RMSE'].append(rmse)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.081e+10, tolerance: 1.034e+08
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.217e+08, tolerance: 1.050e+07
  model = cd_fast.enet_coordinate_descent(


In [9]:
results_df = pd.DataFrame(results_dict)
results_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.842513,22613.475408,27653.628103
1,new_values,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.900554,14813.290347,21699.938249
2,new_values,iter3-OverSampling,NC,LinearRegression,{},0.917646,13818.474604,17443.412938
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 150},0.901458,18787.030384,25107.080582
5,new_values,iter3-OverSampling,SC,Lasso,{'model__alpha': 1.0},0.845874,21346.331258,24982.296867
6,new_values,iter3-OverSampling,VA,LinearRegression,{},0.940935,19224.205151,28133.297029


In [10]:
models_df =  pd.concat([bests_models, results_df], axis=0).sort_values(by=['State','R2'], ascending = [True,False] )
models_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.842513,22613.475408,27653.628103
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.829683,2326.718154,2683.106973
1,new_values,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.900554,14813.290347,21699.938249
1,new_values,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.878805,12267.590107,15930.705726
2,new_values,iter3-OverSampling,NC,LinearRegression,{},0.917646,13818.474604,17443.412938
2,new_values,iter2-cov,NC,LinearRegression,{},0.911957,10052.628324,12590.826233
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
3,all_pca,iter2-cov,NJ,LinearRegression,{},0.595462,31778.200052,35635.501147
4,new_values,iter1-normal,NY,GradientBoosting,{'model__n_estimators': 150},0.912735,80.496891,99.511838
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 150},0.901458,18787.030384,25107.080582


# Undersampling

In [11]:
results_dict = {
    'Version_data': [],
    'Version_iteracion': 'iter3-OverSampling',
    'State': [],
    'Modelo': [],
    'Parameters': [],
    'R2': [],
    'MAE': [],
    'RMSE': []
}
States = bests_models.State.unique()
directory = '../data/final_data/'

for state in States:
    results_dict['Version_data'].append(modelos_dict[state].get('Version_data'))
    results_dict['Modelo'].append(modelos_dict[state].get('Modelo'))
    results_dict['Parameters'].append(modelos_dict[state].get('Parameters'))
    results_dict['State'].append(state)

    filename = f"data_{state}.csv"
    filepath = os.path.join(directory, filename)
    if os.path.exists(filepath):
        data = pd.read_csv(filepath)
        X = data.drop(['MedianPrice', 'State'], axis=1)
        y = data['MedianPrice'] 

        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

        
        # Sobremuestreo utilizando Random Oversampling
        ros = RandomUnderSampler()
        X_train, y_train = ros.fit_resample(X_train, y_train)

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)

        model_name = modelos_dict[state].get('Modelo')
        model = models.get(model_name)

        if model is not None:
            parametros = modelos_dict[state].get('Parameters')

            if isinstance(model, type):
                model.set_params(**parametros)

            # Entrenar el modelo y hacer predicciones
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            # Calcular scores
            r2 = r2_score(y_test, y_pred)
            mae = mean_absolute_error(y_test, y_pred)
            rmse = mean_squared_error(y_test, y_pred, squared=False)

            results_dict['R2'].append(r2)
            results_dict['MAE'].append(mae)
            results_dict['RMSE'].append(rmse)

/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.000e+09, tolerance: 3.447e+07
  model = cd_fast.enet_coordinate_descent(
/usr/local/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 8.217e+08, tolerance: 1.050e+07
  model = cd_fast.enet_coordinate_descent(


In [12]:
results_df = pd.DataFrame(results_dict)
results_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.835018,23620.995350,28303.997296
1,new_values,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.904283,14458.333986,21289.179852
2,new_values,iter3-OverSampling,NC,LinearRegression,{},0.913519,14309.873654,17875.130819
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314
4,new_values,iter3-OverSampling,NY,GradientBoosting,{'model__n_estimators': 150},0.920096,16994.114944,22608.296575
5,new_values,iter3-OverSampling,SC,Lasso,{'model__alpha': 1.0},0.845874,21346.331258,24982.296867
6,new_values,iter3-OverSampling,VA,LinearRegression,{},0.940935,19224.205152,28133.297032


In [13]:
models_df =  pd.concat([models_df, results_df], axis=0).sort_values(by=['State','R2'], ascending = [True,False])
models_df

,Version_data,Version_iteracion,State,Modelo,Parameters,R2,MAE,RMSE
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.842513,22613.475408,27653.628103
0,new_values_log,iter3-OverSampling,FL,GradientBoosting,{'model__n_estimators': 50},0.835018,23620.995350,28303.997296
0,new_values_log,iter2-cov,FL,GradientBoosting,{'model__n_estimators': 50},0.829683,2326.718154,2683.106973
1,new_values,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.904283,14458.333986,21289.179852
1,new_values,iter3-OverSampling,GA,Lasso,{'model__alpha': 1.0},0.900554,14813.290347,21699.938249
1,new_values,iter2-cov,GA,Lasso,{'model__alpha': 1.0},0.878805,12267.590107,15930.705726
2,new_values,iter3-OverSampling,NC,LinearRegression,{},0.917646,13818.474604,17443.412938
2,new_values,iter3-OverSampling,NC,LinearRegression,{},0.913519,14309.873654,17875.130819
2,new_values,iter2-cov,NC,LinearRegression,{},0.911957,10052.628324,12590.826233
3,all_pca,iter3-OverSampling,NJ,LinearRegression,{},0.831319,39046.745229,46977.315314


In [16]:
bests_models_idx = models_df.groupby('State')['R2'].idxmax()
bests_models = models_df.loc[bests_models_idx].groupby('State').first().reset_index()
bests_models

,State,Version_data,Version_iteracion,Modelo,Parameters,R2,MAE,RMSE
0,FL,new_values_log,iter3-OverSampling,GradientBoosting,{'model__n_estimators': 50},0.842513,22613.475408,27653.628103
1,GA,new_values,iter3-OverSampling,Lasso,{'model__alpha': 1.0},0.904283,14458.333986,21289.179852
2,NC,new_values,iter3-OverSampling,LinearRegression,{},0.917646,13818.474604,17443.412938
3,NJ,all_pca,iter3-OverSampling,LinearRegression,{},0.831319,39046.745229,46977.315314
4,NY,new_values,iter3-OverSampling,GradientBoosting,{'model__n_estimators': 150},0.920096,16994.114944,22608.296575
5,SC,new_values,iter3-OverSampling,Lasso,{'model__alpha': 1.0},0.845874,21346.331258,24982.296867
6,VA,new_values,iter3-OverSampling,LinearRegression,{},0.940935,19224.205151,28133.297029


In [13]:
bests_models.to_excel('../data/final_data/bests_models_results.xlsx', index = False)

# Intentando TL 

No se puede, ya que los modelos de ML no son modelos que usan pesos de entrenamiento como las NL. Ademas puede no ser óptimo, ya que los datos de un estado pueden tener características únicas que el modelo haya aprendido a capturar durante el entrenamiento. Por lo tanto, es posible que el rendimiento de las predicciones en un estado diferente no sea tan bueno como en el estado para el cual se entrenó el modelo